In [1]:
%%time
import sys
sys.path.insert(0, '/Users/shuyafeng/PycharmProjects/xiaozaizi/codes/')

import os
import json
import warnings
import pprint


%reload_ext autoreload
%autoreload
%autoreload 2
%config Completer.use_jedi = False

CPU times: user 18.9 ms, sys: 16 ms, total: 34.9 ms
Wall time: 33.5 ms


In [6]:
import numpy as np
import scipy.io
from numpy.random import choice, laplace
from scipy.special import lambertw
from scipy.stats import laplace
from codes.kse_test import kse_test
import pandas as pd

In [7]:
#dataset4 = scipy.io.loadmat('/Users/shuyafeng/PycharmProjects/xiaozaizi/server1.mat')['dataset4']
dataset4 = np.array(pd.read_csv('tr_server_data.csv'))
sizeofD = 100
dataset2 = dataset4[:, :1]
S, _ = np.histogram(dataset2, bins=sizeofD)
eps = np.linspace(0.1, 10, 4)
B = S.reshape(-1, 1)
C = B[:sizeofD]
Output = kse_test(C)

pp = 1. / (Output ** 5)
pp = pp.reshape(-1)
pp /= pp.sum()
NN = 2 ** (15 * Output)
px = np.arange(1, sizeofD+1)

gamma = 0.5
rho = np.exp(lambertw(-gamma / (2 * np.exp(0.5)), k=-1) + 0.5)
rho = np.real(rho)
m = np.log(1 / rho) / (2 * (gamma - rho) ** 2)
k = m * (1 - gamma + rho + np.sqrt(np.log(1 / rho) / (2 * m)))
m = int(float(np.ceil(m)))
k = int(float(np.ceil(k)))


In [8]:
Output

array([0.87756233, 0.31168975, 0.31534626, 0.31313019, 0.31279778,
       0.32487535, 0.32775623, 0.30969529, 0.32354571, 0.29396122,
       0.33617729, 0.32099723, 0.31390582, 0.33573407, 0.30105263,
       0.32587258, 0.3099169 , 0.27168975, 0.2633795 , 0.31024931,
       0.32199446, 0.29484765, 0.33595568, 0.34083102, 0.29440443,
       0.32      , 0.33939058, 0.29030471, 0.33373961, 0.29628809,
       0.32531856, 0.32022161, 0.96354571, 0.29185596, 0.54570637,
       0.32864266, 0.31900277, 0.80288089, 0.90481994, 0.56831025,
       0.95235457, 0.97307479, 0.94393352, 0.86903047, 0.97274238,
       0.9100277 , 0.91689751, 0.97418283, 0.98781163, 0.96299169,
       0.98947368, 0.90094183, 0.94227147, 0.94371191, 0.95168975,
       0.9468144 , 0.97894737, 0.93263158, 0.90592798, 0.98814404,
       0.98836565, 0.91412742, 0.26249307, 0.90393352, 0.29296399,
       0.33240997, 0.27722992, 0.34371191, 0.32587258, 0.32709141,
       0.29130194, 0.32188366, 0.90526316, 0.26648199, 0.31168

In [13]:
EVAL = np.zeros((4, 7))
EVAL1 = np.zeros((4, 7))

for rr in range(4):
    for j in range(1):
        C = B[:sizeofD].reshape(-1)
        GS = np.sort(np.random.choice(C, size=m, replace=False))
        Sensunif = GS[k-1]
        SS = np.sort(np.random.choice(px, size=m, replace=True, p=pp))
        SS = NN[np.floor(SS).astype(int)]
        SS = np.sort(SS)
        Sensanom = SS[k-1]

        unif = laplace.rvs(loc=0, scale=Sensunif/(eps[rr]), size=sizeofD) + C
        noiseanom = laplace.rvs(loc=0, scale=Sensanom/(eps[rr]), size=sizeofD) + NN

        noiseunif = kse_test(unif)
        noiseanom = kse_test(noiseanom)

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if Output[i] > 0.85:
                Index[i, 1] = 1

        ACTUAL = Index
        Origindex = np.where(ACTUAL[:, 1] == 1)[0]

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if noiseunif[i] > 0.85:
                Index[i, 1] = 1

        Uniformm = Index
        Pertindex = np.where(Uniformm[:, 1] == 1)[0]

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if noiseanom[i] > 0.85:
                Index[i, 1] = 1

        Anomm = Index
        Pertindex1 = np.where(Anomm[:, 1] == 1)[0]
        IND1 = Anomm
        IND = Uniformm

        C = len(set(Origindex) | set(Pertindex))
        p = len(Origindex)
        n = sizeofD - p
        N = p + n
        tp = sum(ACTUAL[Origindex, 1] == IND[Origindex, 1])
        tn = N - C
        fp = n - tn
        fn = p - tp
        tp_rate = tp / p
        tn_rate = tn / n
        accuracy = (tp + tn) / N
        sensitivity = tp_rate
        specificity = tn_rate
        precision = tp / (tp + fp)
        recall = sensitivity
        f_measure = 2 * ((precision * recall) / (precision + recall))
        gmean = np.sqrt(tp_rate * tn)
        EVAL1[rr, :] += np.array([accuracy, tp, specificity, precision, recall, f_measure, gmean])

        


In [14]:
import pandas as pd

In [15]:
pd.DataFrame(EVAL1, columns=["accuracy", "tp", "specificity", "precision", "recall", "f_measure", "gmean"])

,accuracy,tp,specificity,precision,recall,f_measure,gmean
0,0.94,25.0,0.971831,0.925926,0.862069,0.892857,7.712507
1,0.63,5.0,0.816901,0.277778,0.172414,0.212766,3.162278
2,0.70,15.0,0.774648,0.483871,0.517241,0.500000,5.333693
3,0.64,25.0,0.549296,0.438596,0.862069,0.581395,5.798335
